In [1]:
import torch
torch.cuda.is_available()

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
from datasets import load_dataset
from datasets import Dataset
dataset = load_dataset('ag_news')
train_ds = dataset['train'].shuffle(seed=42).select([i for i in list(range(5000))])
validation_test_ds = dataset['test'].shuffle(seed=42).select([i for i in list(range(1000))])
validation_ds = Dataset.from_dict(validation_test_ds[0:500])
test_ds = Dataset.from_dict(validation_test_ds[500:1000])





Using custom data configuration default
Reusing dataset ag_news (C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 53.70it/s]
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548\cache-dd0ff9596fea92b0.arrow
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548\cache-12f3c4e4bf422cce.arrow


In [3]:
display(train_ds[0:5])

{'text': ['Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.',
  'Desiring Stability Redskins coach Joe Gibbs expects few major personnel changes in the offseason and wants to instill a culture of stability in Washington.',
  'Will Putin #39;s Power Play Make Russia Safer? Outwardly, Russia has not changed since the barrage of terrorist attacks that culminated in the school massacre in Beslan on Sept.',
  'U2 pitches for Apple New iTunes ads airing during baseball games Tuesday will feature the advertising-shy Irish rockers.',
  'S African TV in beheading blunder Public broadcaster SABC apologises after news bulletin shows footage of American beheaded in Iraq.'],
 'label': [0, 1, 0, 3, 0]}

In [4]:
display(validation_ds[0:5])

{'text': ['Indian board plans own telecast of Australia series The Indian cricket board said on Wednesday it was making arrangements on its own to broadcast next month #39;s test series against Australia, which is under threat because of a raging TV rights dispute.',
  'Stocks Higher on Drop in Jobless Claims A sharp drop in initial unemployment claims and bullish forecasts from Nokia and Texas Instruments sent stocks slightly higher in early trading Thursday.',
  'Nuggets 112, Raptors 106 Carmelo Anthony scored 30 points and Kenyon Martin added 24 points and 16 rebounds, helping the Denver Nuggets hold off the Toronto Raptors 112-106 Wednesday night.',
  'Stocks Higher on Drop in Jobless Claims A sharp drop in initial unemployment claims and bullish forecasts from Nokia and Texas Instruments sent stocks higher in early trading Thursday.',
  'REVIEW: \'Half-Life 2\' a Tech Masterpiece (AP) AP - It\'s been six years since Valve Corp. perfected the first-person shooter with "Half-Life." 

In [5]:
display(test_ds[0:5])

{'text': ['A Sneak Peek at Trillian 3.0 Instant Messaging The popular IM consolidation service adds audio and video chat.',
  'New Technology Powers Fuel Cells A new fuel cell for notebook PCs, more compact and powerful than competing technologies, could be on the market in early 2006 at a price of around \\$90, its Japanese inventors claim.',
  "Court rules against state Web-blocking law A Pennsylvania law requiring Internet service providers to block Web sites deemed by the state's prosecuting attorneys to be child pornography has been reversed by a U.S. federal court on free-speech grounds.",
  'Braves, Smoltz agree to new deal Atlanta, GA (Sports Network) - The Atlanta Braves announced Thursday that the team has come to terms with longtime pitcher John Smoltz to a new two-year contract with a club option for the 2007 season.',
  "US Supreme Court asked to rule on homosexuals' right to adopt children (AFP) AFP - The American Civil Liberties Union, the leading US civil rights group, 

In [6]:
from transformers import set_seed

set_seed(42)

In [7]:
#defining model:
from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, 
                          Trainer, EarlyStoppingCallback,
                          TrainingArguments)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenize_func = lambda sentences: tokenizer(sentences['text'], 
                                            padding='max_length', truncation=True, max_length = 200)



train_dataset = train_ds.map(tokenize_func, batched=True)
val_dataset = validation_ds.map(tokenize_func, batched=True)
test_dataset = test_ds.map(tokenize_func, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

Parameter 'function'=<function <lambda> at 0x0000020C6C1D9D30> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Loading cached processed dataset at C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548\cache-1c80317fa3b1799d.arrow
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 34.57ba/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', '

In [8]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average='macro')["f1"]
   return {"accuracy": accuracy, "f1": f1}


training_args = TrainingArguments(
    output_dir=r'C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\original_model',          # output directory
    num_train_epochs=20,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
    
    logging_dir=r'C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\logs\original_model',            # directory for storing logs
    save_total_limit = 1,
    load_best_model_at_end=True,
    #metric_for_best_model='f1',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
)


trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [9]:
#training:
trainer.train(ignore_keys_for_eval=['last_hidden_state', 'hidden_states', 'attentions'])
# We are going to get multiple loss values on each training step here


The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 3000
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 940


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.390400,1.368018,0.318000,0.220496
2,1.324700,1.221997,0.788000,0.792612
3,1.010200,0.775559,0.810000,0.809491
4,0.595200,0.497714,0.864000,0.864526
5,0.390800,0.405891,0.878000,0.878727
6,0.311600,0.391457,0.880000,0.880664
7,0.254000,0.388851,0.890000,0.892059
8,0.206800,0.379693,0.886000,0.886861
9,0.157400,0.331542,0.892000,0.893797
10,0.101000,0.383789,0.892000,0.894066


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32
Using the latest cached version of the module from C:\Users\DELL\.cache\huggingface\modules\datasets_modules\metrics\f1\6a64529c5745513ceb230ce9696115dcad6ae0fedad9946e3f2f723a65a0cd53 (last modified on Sat Aug 13 21:10:53 2022) since it couldn't be found locally at f1, or remotely on the Hugging Face Hub.
Saving model checkpoint to C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\original_model\checkpoint-47
Configuration saved in C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\original_model\checkpoint-47\config.json
Model weights saved in C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\original_model\checkpoint-47\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `Disti

Using the latest cached version of the module from C:\Users\DELL\.cache\huggingface\modules\datasets_modules\metrics\f1\6a64529c5745513ceb230ce9696115dcad6ae0fedad9946e3f2f723a65a0cd53 (last modified on Sat Aug 13 21:10:53 2022) since it couldn't be found locally at f1, or remotely on the Hugging Face Hub.
Saving model checkpoint to C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\original_model\checkpoint-423
Configuration saved in C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\original_model\checkpoint-423\config.json
Model weights saved in C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\original_model\checkpoint-423\pytorch_model.bin
Deleting older checkpoint [C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\original_model\checkpoint-376] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Run

TrainOutput(global_step=564, training_loss=0.48898827141903817, metrics={'train_runtime': 968.403, 'train_samples_per_second': 61.958, 'train_steps_per_second': 0.971, 'total_flos': 4768996442112000.0, 'train_loss': 0.48898827141903817, 'epoch': 12.0})

In [10]:
#save model:
trainer.save_state()
trainer.save_model()

Saving model checkpoint to C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\original_model
Configuration saved in C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\original_model\config.json
Model weights saved in C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\original_model\pytorch_model.bin


In [11]:
#evaluate:
trainer.evaluate(test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32


{'eval_loss': 0.3214978575706482,
 'eval_accuracy': 0.9,
 'eval_f1': 0.8979683758897417,
 'eval_runtime': 37.2709,
 'eval_samples_per_second': 13.415,
 'eval_steps_per_second': 0.429,
 'epoch': 12.0}